In [410]:
import random
import math
import copy
from random import uniform, randint, shuffle, sample

import time

In [411]:
def encoding_population(initial_population):
    dict = {
        'a' : 0,
        'c' : 1,
        'g' : 2,
        't' : 3
    }
    l=[]
    for i in initial_population:
        k=''
        for j in i:
            k+=str(dict[j])
        l.append(k)
    return l

In [412]:
def insert_symbol(src_string,inserted_string,pos):
    return ''.join(src_string[:pos] + inserted_string + src_string[pos:])
# Given set of strings and population size for SCS problem

# supersequence_generate(['acg', 'cat', 'gtt','tgc'])
def supersequence_generate(set_of_strings):

    '''
        Make a copy of the set_of_strings parameter for maintaining the original
        set
    '''
    copied_set_of_strings = copy.deepcopy(set_of_strings)
    supersequence = ''.join(copied_set_of_strings.pop(random.randint(0,len(set_of_strings)-1)))

    for i in range(len(copied_set_of_strings)):
        # print("i = ",i)
        counter = 0
        for j in copied_set_of_strings[i]:
            inserted_pos = random.randint(counter,len(supersequence))
            # print("j and counter and supersequence length and inserted index",j," ",counter," ",len(supersequence)," ",inserted_pos)
            if inserted_pos == len(supersequence) or j != supersequence[inserted_pos]:
                supersequence=insert_symbol(supersequence,j,inserted_pos)
            counter = inserted_pos +1
            # print(supersequence)

    return supersequence
def population_generation(pop_size,set_of_strings):
    # l=[]
    # for i in range(pop_size):
    #     l.append(supersequence_generate(set_of_strings))
    # return l
    population = []

    while len(population) < pop_size:
        population.append(supersequence_generate(set_of_strings))

    # # Find the length of the string that appears the most
    # most_common_length = max(len(string) for string in population)

    # # Remove strings that do not match the most common length
    # population = [string for string in population if len(string) == most_common_length]

    # # Continue generating strings until the length of the container equals the original population size
    # while len(population) < pop_size:
    #     generated_supersequence = supersequence_generate(set_of_strings)
    #     if(len(generated_supersequence)  == most_common_length):
    #         population.append(generated_supersequence)

    return population
def initialization(pop_size,set_of_strings):
    initial_population =  population_generation(pop_size,set_of_strings)

    encoded_population = encoding_population(initial_population)


    return encoded_population
molecules = []
origin=initialization(10000,['acg', 'cat', 'gtt','tgc'])
#initializa a list of subsequence that are encoded bases on the encoding_population function
for original_string   in origin:
    digit_list = [int(digit) for digit in original_string]
    molecules.append(digit_list)
for mol in molecules :
    print(mol)

Streaming output truncated to the last 5000 lines.
[3, 0, 2, 1, 1, 0, 2, 3, 2, 3, 3]
[2, 0, 1, 0, 2, 3, 3, 3, 2, 1]
[2, 3, 0, 1, 1, 3, 2, 3, 0, 3, 1]
[3, 2, 1, 0, 1, 2, 1, 0, 3, 2, 3, 3]
[2, 3, 1, 0, 3, 2, 0, 1, 2, 1, 3, 3]
[1, 0, 3, 2, 3, 0, 1, 3, 2, 2, 1, 3]
[0, 2, 1, 3, 2, 3, 3, 1, 2, 0, 3]
[2, 3, 3, 0, 1, 3, 1, 2, 0, 3, 2, 1]
[2, 3, 1, 0, 3, 2, 1, 2, 1]
[0, 1, 2, 3, 2, 1, 3, 2, 3, 0, 1, 3]
[3, 1, 2, 3, 0, 2, 3, 1, 3, 2]
[1, 0, 3, 1, 2, 2, 3, 3, 1]
[3, 2, 0, 3, 1, 1, 3, 2, 0, 1, 3, 2]
[0, 1, 0, 3, 1, 2, 2, 3, 3, 1]
[2, 3, 0, 1, 3, 3, 1, 2, 2, 0, 1, 3]
[2, 3, 1, 2, 0, 1, 3, 2, 3]
[0, 3, 1, 2, 0, 2, 3, 1, 3, 2]
[1, 0, 3, 2, 2, 1, 1, 3, 3, 2]
[0, 1, 2, 2, 3, 3, 0, 3, 2, 1]
[0, 1, 1, 0, 2, 3, 3, 2, 3, 2, 1]
[0, 1, 3, 2, 0, 3, 2, 1, 3]
[1, 3, 2, 2, 1, 0, 1, 3, 2, 3, 3]
[0, 1, 1, 0, 2, 3, 2, 3, 1]
[2, 3, 2, 0, 1, 1, 0, 3, 3, 3, 2]
[1, 3, 2, 0, 1, 2, 2, 3, 3, 1, 0, 3]
[3, 0, 1, 1, 2, 2, 1, 2, 0, 3, 3]
[0, 1, 0, 2, 3, 3, 2, 1, 1, 2, 3]
[2, 0, 1, 2, 1, 0, 3, 3, 2, 1]
[0, 1, 1, 2, 0, 3, 2, 3,

In [413]:
# The Ant Colony Optimization (ACO) algorithm simulates the foraging behavior of ants to find an optimal path.
# In this algorithm, a predefined number of ants explore possible paths, leaving behind a pheromone trail.
# The intensity of the pheromone trail is influenced by the number of ants that traverse a particular path.
# Paths with a higher concentration of pheromones become more attractive to subsequent ants, increasing the likelihood of those paths being chosen as optimal routes.
# For instance, if multiple ants traverse a path with different distances, the algorithm calculates the total
#  pheromone level on that path, reflecting the cumulative choices of all ants that have moved along it. This collective information guides the algorithm in identifying promising paths for the final solution.
# Approach to Solve SCS Problem using ACO Algorithm:

# Step 1: Encode Subsequences
# - Encode each subsequence as a number (0 to 3), representing different elements.

# Step 2: Create 2D List
# - Form a 2D list where each row corresponds to the string value of an encoded subsequence.

# Step 3: Apply ACO Algorithm
# - Utilize the Ant Colony Optimization algorithm on the 2D list, treating each row as a node.
# - Find the most optimal path through these nodes, representing a solution to the SCS problem.

# Step 4: Translate Optimal Path
# - Translate the obtained optimal path back into the original sequence of 0 to 3.

# Step 5: Output Optimal SCS
# - Construct the most optimal Shortest Common Supersequence (SCS) using the translated path.
a = time.time()

# Split the matrix so that it fits the condition that the column number is larger than the row number
distance_matrix =list(map(list, zip(*molecules)))
# for m in distance_matrix :
#     print(m)
# print(len(distance_matrix[0]))
n = len(distance_matrix)

# sumarize the total cost walk by an ant in the given path
def calculate_distance(ant_path: list) -> float:
    current_index = ant_path[0]  # Calculation of distance between nodes
    distance = 0
    for next_index in ant_path[1:]:
        # print(current_index)
        # print(next_index)
        distance += distance_matrix[current_index][next_index]
        current_index = next_index
    return distance  # Distance returned

# swap a given sequence
def swap(sequence: list, i: int, j: int) -> None:
    temp = sequence[i]  # Node swapping function
    sequence[i] = sequence[j]
    sequence[j] = temp

# update a single path that the ant have walk then return the updated calculate_distance
def local_pheromone_update(ant_path: tuple, a: int, b: int) -> tuple:
    updated_ant_path = ant_path[0][:]
    swap(updated_ant_path, a, b)
    return (updated_ant_path, calculate_distance(updated_ant_path))  # Return the ant path.
# update multiple path that the ant have walk then return the updated calculate_distance
def global_pheromone_update(ant_path: tuple, a: int, b: int, c: int) -> tuple:
    updated_ant_path = ant_path[0][:]
    swap(updated_ant_path, a, b)
    swap(updated_ant_path, b, c)
    return (updated_ant_path, calculate_distance(updated_ant_path))  # Return the ant path.

while True:
    num_ants = 10  # Number of ants
    # worst path or good path or determin by a poprebility if an worst ant have a hige chances of taking a path with low pop while good ant do the oppersite
    worst_ants = int(0.1 * num_ants)  # ant that take a worst path

    best_ants = int(0.9 * num_ants)  # Good-value ants  , ant that take a good path(less costly)

    # alpha and beta are just constance that can  be change to see different result
    alpha = 2.0  # Alpha value required for the transition method (float)
    beta = 2.0  # Beta value required for the transition method (float)
    pass_max = 15  # Transition method variables (increased pass_max for a higher range)
    pass_min = 0  # Transition method variables
    transition_probability = 0.5  # Float value the poperties in which the ants will transition
    pass_method = alpha * 1 / n * beta * (pass_max - pass_min)  # Float value
    iteration_size = 100  # How many times the main loop will run
    ants = []  # Array (list) for ants
    initial_path = list(range(0, n))
    for i in range(num_ants):
        # Generate the initial path, meaning that it randomly generates path, order in which each row is selected
        ant_path = sample(initial_path, n)
        # print(ant_path)
        ants.append((ant_path, calculate_distance(ant_path)))

    # Sort the second element of the structure, i.e., the distance in which the ant will travel
    ants.sort(key=lambda x: x[1])
    # The main loop in the program
    for iteration_index in range(iteration_size):
        go_ant = ants[randint(0, best_ants)]  # Movement group to be selected
        random_ant_index = randint(0, int(passMethod))  # The next group will be shaped according to the transition method
        # The transition probability , the probability in which an ant will choose a path
        # the random.random function is an number between [0,1] it decide if the ant sort change it operation
        if random.random()  < transition_probability:

            more_powerful_ant = local_pheromone_update(go_ant, randint(0, n-1), randint(0, n-1))
            if ants[random_ant_index][1] > more_powerful_ant[1]:
                ants[random_ant_index] = more_powerful_ant
        else:
            for i in range(num_ants - worst_ants, num_ants):
                ants[i] = local_pheromone_update(ants[i], randint(0, n-1), randint(0, n-1))
            ants.sort(key=lambda x: x[1])

        # get the ant with the lowest cost ie the path that the ant walk with the lowest cost
        low_cost_ant = ants[0]
        effectively_global_ant = global_pheromone_update(
            low_cost_ant, randint(0, n-1), randint(0, n-1), randint(0, n-1))
        # make a goble pheromon udpate so that it can check again if the lowest cost ant that have global_pheromone_update distince is smaller than the ant which does not
        if ants[0][1] > effectively_global_ant[1]:
            ants[0] = effectively_global_ant
        ants.sort(key=lambda x: x[1])
    # Variable to store the cost that the ant will take
    ant_costs = []
    # For loop for adding the cost that the ant will take
    for i in range(len(ants[0][0]) - 1):
        current_city = ants[0][0][i]
        next_city = ants[0][0][i + 1]
        cost = molecules[current_city][next_city]
        ant_costs.append(cost)
    unique_elements = set(ant_costs)
    # Condition to break if the ant has taken a path with all variable words in that are contained in the initialization, i.e., ['acg', 'cat', 'gtt','tgc']
    if set([0, 1, 2, 3]).issubset(unique_elements):
        break

print("SCS", ant_costs)


SCS [0, 3, 3, 1, 3, 3, 2]
